In [1]:
from tool import *
from tqdm import tqdm

In [7]:
os.chdir('D:/Quanta/Quanta PDR') # PDR 이 저장되어 있는 폴더로 이동
dr_list = os.listdir() # PDR 폴더내의 각 모델별 하위 폴더들을 리스트로 저장

# PDR 디렉토리 하위 디렉토리를 차례로 돌면서 가장 최근의 PDR 파일로 대표모델의 전체 Suffix에 대해 PDR 정보를 데이티프레임화한후, CSV로 저장함
for dr in tqdm(dr_list):
    os.chdir('D:/Quanta/Quanta PDR/'+dr)
    filelist = os.listdir()
    df = make_all_PDR_data_from_certain_model(filelist[-1]) # 각 모델별 폴더의 가장 마지막 파일(가장 최신 파일)에서 PDR 정보를 만듬
#     df.to_csv('D:/Python result/Quanta PDRDB/'+dr+'.csv', encoding = 'cp949')
    df.to_csv('D:/Python result/Quanta PDRDB/'+dr+'.csv')
print(f"Updated Time : {datetime.datetime.today().strftime('%Y-%m-%d-%H:%M')}")

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [03:36<00:00, 13.52s/it]

Updated Time : 2023-04-27-09:17


In [8]:
df = pd.DataFrame()
os.chdir('D:/Python result/Quanta PDRDB/')
filelist = os.listdir()
for file in tqdm(filelist):
#     temp = pd.read_csv(file, encoding='cp949')
    temp = pd.read_csv(file)
    df = pd.concat([df, temp])

df.insert(0, 'Model', df['Model.Suffix'].apply(lambda x:x.split('-')[0]))
df.insert(0, 'Series', df['Model'].replace(srt_model))
df.reset_index(inplace=True, drop=True)
df.to_excel('D:/Python result/Quanta PDR database.xlsx', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 41.39it/s]


### 신규 PDR을 기존 PDR 엑셀파일에 추가

In [2]:
import win32com.client

In [3]:
excel = win32com.client.Dispatch('Excel.Application')
excel.Visible = True

In [4]:
os.chdir('D:/Downloads/PDRs') # 복사할 PDR이 들어있는 폴더로 이동한 후 PDR 파일을 리스트로 만듬
copy_list = os.listdir()
copy_list

['PDR DETAIL(15U40Q-G AA5HK).xlsx', 'PDR DETAIL(15U40Q-G AR5MK).xlsx']

In [5]:
def change_model_name(name):
    try:
        return srt_model[name]
    except:
        return name

In [6]:
for i, file in enumerate(copy_list):
    wb1 = excel.Workbooks.Open('D:/Downloads/PDRs' + '/' + copy_list[i])
    ws1 = wb1.Activesheet
    folder_name = change_model_name(copy_list[i].split(' ')[1].split('(')[1].split('-')[0]) +'_' + supplier_model_map[change_model_name(copy_list[i].split(' ')[1].split('(')[1].split('-')[0])]
    file_path = 'D:/Quanta/Quanta PDR/' + folder_name
    os.chdir(file_path)
    filename2 = os.listdir()[-1]
    wb2 = excel.Workbooks.Open(file_path + '/' + filename2)
    ws1.Copy(Before=wb2.Worksheets('List'))
    wb1.Close()
    wb2.Close()